# Transportation Problem 

For a sequence of steps starting from 1 to n, you can either walk or take a tram, subject to following rules:
<ul> 
    <li> Tram results in distance from <code>s </code>to <code> 2s </code> cost. </li>
    <li> Walking will result in distance from <code> s </code> to <code> s + 1 </code> cost.</li>
</ul>

Return the optimal sequence of steps from 1 to any given n, that would result in minimum cost.

## Class Blueprint

Our search problem will have a blueprint of the following. The implementation is generic, it need have the following structure.

In [42]:
class TransportationProblem:
    
    def __init__(self, n):
        """
            Initializes an instance of the transportation problem with the provided upper bound.
            
            Args:
                n (int): the upper bound in the number of steps.  
        """
        
        self.n = n
    
    def start(self):
        """
            Returns the start state of the problem.
        """
        
        return 1
    
    def is_goal(self, s):
        """
            Checks whether the provided state is the goal state or not.
            
            Args:
                s (int) : the provided state.
            
            Returns:
                bool : Whether it is the goal state or not.
        """
        
        if s == self.n:
            return True
        
        return False
    
    def actions(self, s):
        """
            Returns the actions that can be taken at the subtree rooted at the node 's'. Actions in this instance of the
            problem are either 'walk' or 'tram'.
            
            Args:
                s (int) : The state at which we want to check the available actions.
            
            Returns:
                actions (list) : The available actions at the state 's'.
        """
        
        actions = []
        
        if s + 1 <= self.n:
            actions.append('walk')
        if s * 2 <= self.n:
            actions.append('tram')
        
        return actions
    
    def cost(self, a):
        """
            Computes the cost associated with the action at the given state.
            
            Args:
                a(str) : The action to take at the given state.
            
            Returns:
                int : The cost associated with that action at the state s.
        """
        
        if a == 'tram':
            return 2
        elif a == 'walk':
            return 1
    
    def transition(self, s, a):
        """
            Returns the transition step for the action provided at the given state.
            
            Args:
                s(int) : The state provided.
                a(str) : The action to take at the given state.
            
            Returns:
                int : The transition associated with that action at the state s.
        """
        
        if a == 'tram':
            return 2*s
        elif a == 'walk':
            return s+1
        
        return 0

## Backtracking Search - Brute Force Algorithm
This algorithm computes the maximal effecient low-cost steps using the backtrack search. Note that this is not computationally effecient at large values of <code>n</code> of problem.

We will be passing the instance of the problem to the function. The code blueprint is as follows:

<code>def backtrack_search(prob)</code>

In [43]:
def backtrack_search(prob):
    from math import inf
    min_cost = inf
    history = []
    
    def recurse(s, c, hist):
        nonlocal min_cost, history
        
        if prob.is_goal(s):
            if c < min_cost:
                min_cost = c
                history = hist
        else:
            actions = prob.actions(s)
            
            for act in actions:
                new_s = prob.transition(s,act)
                cost = prob.cost(act)
                
                recurse(new_s, cost+c, hist + [(s, act, new_s)])
    
    recurse(prob.start(), 0, [])
    return min_cost, history

In [44]:
prob =TransportationProblem(6)
print(backtrack_search(prob))

(4, [(1, 'walk', 2), (2, 'walk', 3), (3, 'tram', 6)])


## Backtracking Search - Dynamic Programming Algorithm
The issue with the recursive call of the brute force algorithm is that for very large values of <code>n</code>, the call stack for the recursion will overflow. The reason behind this is that for very large values of <code>n</code>, say <code>n=1000</code>, the branches of the tree (_which basically represent actions at that node_) grow exponentially.

We use dynamic programming to solve this problem. The function prototype remains the same as above.

In [45]:
def backtrack_search_dp(prob):
    """
        Backtrack search with dynamic programming.
        
        Args:
            prob (TransportationProblem): The transportation problem to solve.

        Returns:
            cost (int): The minimum cost to reach the goal state.
            history (list): The sequence of actions that leads to the goal state with the minimum cost of the goal found.
    """
    min_cost = float('inf')
    min_history = []
    cache = [float('inf')] * (prob.n)
    
    def recurse(s, cost, history):
        nonlocal min_cost, min_history,cache  # these variables are out of the scope of the function so nonlocal is added
        
        if prob.is_goal(s):
            if cost < min_cost:
                min_cost = cost
                min_history = history
            return min_cost, min_history
        elif  cache[s] <= cost:
            return min_cost, min_history
        else:
            cache[s] = cost
            for action in prob.actions(s):
                new_state = prob.transition(s, action)
                new_cost = cost + prob.cost( action)
                min_cost, min_history = recurse(new_state, new_cost, history + [(s, action, new_state)])
            return min_cost, min_history

    min_cost, min_history = recurse(prob.start(), 0, [])

    return min_cost, min_history

In [46]:
prob = TransportationProblem(1000)
print(backtrack_search_dp(prob))

(22, [(1, 'walk', 2), (2, 'walk', 3), (3, 'tram', 6), (6, 'walk', 7), (7, 'tram', 14), (14, 'walk', 15), (15, 'tram', 30), (30, 'walk', 31), (31, 'tram', 62), (62, 'tram', 124), (124, 'walk', 125), (125, 'tram', 250), (250, 'tram', 500), (500, 'tram', 1000)])


## Depth First Search
The depth first search algorithm is the same as backtrack search, but it does not optimise it. Rather what it does is that it stops the recursion on the first instance of the goal found. In other words, we are not concerned with the cost, but just the goal.

The prototype is <code>def depth_first(prob)</code>.

In [47]:
def depth_first_recursive(prob):
    """
    Performs a depth-first search on the transportation problem.

    Args:
        prob (TransportationProblem): The transportation problem to solve.

    Returns:
        cost (int): The cost to reach the goal state.
        history (list): The sequence of actions that leads to the goal state with the first instance of the goal found.
    """
    min_cost = float('inf')
    min_history = []
    cache = [float('inf')] * (prob.n)
    is_goal_found = False
    
    def dfs(s, cost, history):
        nonlocal min_cost, min_history, is_goal_found
        
        if prob.is_goal(s):
            if cost < min_cost:
                min_cost = cost
                min_history = history
                is_goal_found = True
            return
        
        if cost >= cache[s] or is_goal_found:
            return
        
        cache[s] = cost
        
        for action in prob.actions(s):
            new_state = prob.transition(s, action)
            new_cost = cost + prob.cost(action)
            dfs(new_state, new_cost, history + [(s, action, new_state)])
    
    dfs(prob.start(), 0, [])
    
    return min_cost, min_history

In [48]:
prob = TransportationProblem(1000)
print(depth_first_recursive(prob)) # 501, because the recursion is being called at the left-left branch of the tree and will finally stop.

(501, [(1, 'walk', 2), (2, 'walk', 3), (3, 'walk', 4), (4, 'walk', 5), (5, 'walk', 6), (6, 'walk', 7), (7, 'walk', 8), (8, 'walk', 9), (9, 'walk', 10), (10, 'walk', 11), (11, 'walk', 12), (12, 'walk', 13), (13, 'walk', 14), (14, 'walk', 15), (15, 'walk', 16), (16, 'walk', 17), (17, 'walk', 18), (18, 'walk', 19), (19, 'walk', 20), (20, 'walk', 21), (21, 'walk', 22), (22, 'walk', 23), (23, 'walk', 24), (24, 'walk', 25), (25, 'walk', 26), (26, 'walk', 27), (27, 'walk', 28), (28, 'walk', 29), (29, 'walk', 30), (30, 'walk', 31), (31, 'walk', 32), (32, 'walk', 33), (33, 'walk', 34), (34, 'walk', 35), (35, 'walk', 36), (36, 'walk', 37), (37, 'walk', 38), (38, 'walk', 39), (39, 'walk', 40), (40, 'walk', 41), (41, 'walk', 42), (42, 'walk', 43), (43, 'walk', 44), (44, 'walk', 45), (45, 'walk', 46), (46, 'walk', 47), (47, 'walk', 48), (48, 'walk', 49), (49, 'walk', 50), (50, 'walk', 51), (51, 'walk', 52), (52, 'walk', 53), (53, 'walk', 54), (54, 'walk', 55), (55, 'walk', 56), (56, 'walk', 57), (5

## Breadth First Search
The Breadth first search algorithm explores each vertex of the tree level by level and then returns the cost. Intuitively, we can say that the cost is being incremented by 1, when each level of the tree is being incremented.

In [49]:
def bfs_search(prob):
  """
  Performs a breadth-first search on the transportation problem.

  Args:
    prob (TransportationProblem): The transportation problem to solve.

  Returns:
    min_cost (int): The minimum cost to reach the goal state.
    min_history (list): The sequence of actions that leads to the goal state with the minimum cost.
  """

  queue = [(prob.start(), 0, [])]
  visited = set()

  while queue:
    s, cost, history = queue.pop(0)

    if prob.is_goal(s):
      return cost, history

    if s not in visited:
      visited.add(s)

      for action in prob.actions(s):
        new_state = prob.transition(s, action)
        new_cost = cost + prob.cost(action)
        new_history = history + [(s, action, new_state)]

        queue.append((new_state, new_cost, new_history))

  return float('inf'), []

In [50]:
prob = TransportationProblem(10)
levels, actions = bfs_search(prob)

print(f'The levels of the bfs tree are {levels}')
      
print(f'The steps leading to it are {actions}')

The levels of the bfs tree are 6
The steps leading to it are [(1, 'walk', 2), (2, 'tram', 4), (4, 'walk', 5), (5, 'tram', 10)]


## Iterative Deepening Search

Iterative deepening search (IDS) is a search algorithm that combines the completeness of breadth-first search (BFS) with the space efficiency of depth-first search (DFS). It works by repeatedly performing DFS searches with increasing depth limits until the goal state is found.

In [51]:
def iterative_search(prob):
  """
  Performs an iterative deepening search on the transportation problem.

  Args:
    prob (TransportationProblem): The transportation problem to solve.

  Returns:
    min_cost (int): The minimum cost to reach the goal state.
    min_history (list): The sequence of actions that leads to the goal state with the minimum cost.
  """

  depth_limit = 0

  while True:
    min_cost, min_history = dls_search(prob, depth_limit)

    if min_cost != float('inf'):
      return min_cost, min_history

    depth_limit += 1

def dls_search(prob, depth_limit):
  """
  Performs a depth-limited search on the transportation problem with the given depth limit.

  Args:
    prob (TransportationProblem): The transportation problem to solve.
    depth_limit (int): The depth limit.

  Returns:
    min_cost (int): The minimum cost to reach the goal state within the given depth limit.
    min_history (list): The sequence of actions that leads to the goal state with the minimum cost within the given depth limit.
  """

  stack = [(prob.start(), 0, [])]
  visited = set()

  while stack:
    s, cost, history = stack.pop()

    if prob.is_goal(s):
      return cost, history

    if cost > depth_limit or s in visited:
      continue

    visited.add(s)

    for action in prob.actions(s):
      new_state = prob.transition(s, action)
      new_cost = cost + prob.cost(action)
      new_history = history + [(s, action, new_state)]

      stack.append((new_state, new_cost, new_history))

  return float('inf'), []


In [52]:
print(iterative_search(prob))

(7, [(1, 'tram', 2), (2, 'tram', 4), (4, 'walk', 5), (5, 'tram', 10)])
